In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [ ]:
import vso.util
import vso.data

OBJ_NAME='SA20'
SESSION_TAG='2024/20241003'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vso.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
settings = vso.util.Settings(session_layout.settings_file_path)

In [ ]:
import vso.phot

BANDS = ('B', 'V')
provider = vso.phot.BatchDataProvider(session_layout)
data = provider.batch_and_sequence_band_pair(BANDS)

data['C'] = data[BANDS[0]]['mag'] - data[BANDS[1]]['mag']
data[f'd{BANDS[0]}'] = data[BANDS[0]]['mag'] - data[f'instr {BANDS[0]}']['mag']
data[f'd{BANDS[1]}'] = data[BANDS[1]]['mag'] - data[f'instr {BANDS[1]}']['mag']
batched = data.group_by('batch_id').groups
batch = batched[0]
# & (data['auid'] != '000-BMM-136') & (data['auid'] != '000-BMM-152')

band = 'B'
batch


In [ ]:
from scipy import stats as sst
r1 = sst.linregress(batch[band]['mag'], batch[f'instr {band}']['mag'])
r1


In [ ]:
ci = batch['C']
dA = batch[f'd{band}']
rC = sst.linregress(ci,dA )
# T=rC.slope
[ sst.linregress(batch['C'], batch[f'd{band}']).slope for batch in batched]
# ci, dA

In [ ]:
sst.linregress(batch[band]['mag'], batch[f'instr {band}']['mag'] + rC.slope*batch['C'])


In [ ]:

import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(12.80, 6.00), layout="constrained")
gs = fig.add_gridspec(1, 2)

ax = fig.add_subplot(gs[0, 0])
for star in batch:
    ax.scatter(star[band]['mag'], star[f'instr {band}']['mag'], label=star['auid'])
ax.set_xlabel(band)
ax.set_ylabel(band.lower())

ax = fig.add_subplot(gs[0, 1])
for star in batch:
    ax.scatter(star['C'], star[f'd{band}'], label=star['auid'])
ax.set_xlabel(f'{BANDS[0]} - {BANDS[1]}')
ax.set_ylabel(f'{band} - {band.lower()}')

fig.legend(loc='outside right')

plt.show()